 Cointegration Analysis of GLD and GDX
 ======================================
 This notebook downloads historical price data for SPDR Gold Shares (GLD) and VanEck Gold Miners ETF (GDX),
 performs a cointegration analysis using:
   1. The Johansen Test
   2. The Augmented Dickey-Fuller (ADF) Test on residuals from OLS regression
   +
   0
 These tests help determine whether a mean-reverting spread exists, which is useful for pair trading.

In [28]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from ib_insync import *
from datetime import datetime, timedelta
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import yfinance as yf
from arch.unitroot import VarianceRatio

In [29]:
# lets look at the prices from both to visually see cointegration before testing:

# download data from yfinance as TWS is down at the time of writing this
start_date = '2013-01-01'
end_date = '2017-12-31'

print("Downloading GLD data...")
gld = yf.download('GLD', start=start_date, end=end_date, auto_adjust=True)
print("Downloading GDX data...")
gdx = yf.download('GDX', start=start_date, end=end_date, auto_adjust=True)

# rename columns
gld_close = gld[['Close']].copy()
gld_close.columns = ['GLD']

gdx_close = gdx[['Close']].copy()
gdx_close.columns = ['GDX']

# concatenate the data for easier management
pair = pd.concat([gld_close, gdx_close], axis=1).dropna()
print("\nCombined data:")
print(pair.head())

# plot prices so we can visualize relationship
print("\nGenerating plot...")
fig = go.Figure()
fig.add_trace(go.Scatter(x=pair.index, y=pair['GLD'], mode='lines', name='GLD Close'))
fig.add_trace(go.Scatter(x=pair.index, y=pair['GDX'], mode='lines', name='GDX Close'))

fig.update_layout(
    title='GLD and GDX Closing Prices (2013-2017)',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Combined data:
                   GLD        GDX
Date                             
2013-01-02  163.169998  41.983818
2013-01-03  161.199997  40.236713
2013-01-04  160.440002  40.406078
2013-01-07  159.429993  39.648411
2013-01-08  160.559998  39.799942

Generating plot...


In [30]:
# now lets prove that these two price series are cointegrated:

# cointegration test method
def cointegration_test(series1, series2):
    # step 1: Regress series1 on series2
    X = sm.add_constant(series2)
    model = sm.OLS(series1, X).fit()
    beta = model.params[1]
    spread = series1 - beta * series2
    
    # step 2: ADF test on residuals
    adf_result = adfuller(spread.dropna())
    print(f"ADF Statistic: {adf_result[0]}")
    print(f"p-value: {adf_result[1]}")
    print(f"Critical Values: {adf_result[4]}")

    return beta, spread, adf_result[1]

def hurst_exponent(series):
    """Calculates Hurst exponent for mean-reversion detection"""
    lags = range(2, 100)
    tau = [np.std(np.subtract(series[lag:], series[:-lag])) for lag in lags]
    poly = np.polyfit(np.log(lags), np.log(tau), 1)
    return poly[0]

def enhanced_cointegration_test(series1, series2):
    """Comprehensive cointegration check with diagnostics"""
    # use Engle-Granger cointegration test for pairwise cointegration
    beta, spread, p_value = cointegration_test(series1, series2)

    # do they cointegrate within a 95% confidence range?
    if p_value < 0.05:
        print("Cointegration exists (reject null hypothesis)")
    else:
        print("No cointegration - find different pair")
        
    # calculate Hurst exponent
    H = hurst_exponent(spread.values)
    
    # calculate half-life
    lagged_spread = spread.shift(1).dropna()
    delta = spread.diff().dropna()
    model = sm.OLS(delta, lagged_spread)
    half_life = -np.log(2) / model.fit().params[0]
    
    # run variance ratio test
    vr = VarianceRatio(spread).pvalue
    
    print(f"\nAdditional Diagnostics:")
    print(f"Hurst Exponent: {H:.4f} → {'Mean-reverting' if H<0.5 else 'Trending'}")
    print(f"Half-life: {half_life:.2f} days")
    print(f"Variance Ratio p-value: {vr:.4f}")
    
    return beta, spread, p_value, H, half_life

# run cointegration test:
beta, spread, p_value, H, half_life = enhanced_cointegration_test(pair['GLD'], pair['GDX'])

z_score = (spread - spread.mean()) / spread.std()

ADF Statistic: -3.0636911526322304
p-value: 0.029359521342377376
Critical Values: {'1%': np.float64(-3.4356006420838963), '5%': np.float64(-2.8638586845641063), '10%': np.float64(-2.5680044958343604)}
Cointegration exists (reject null hypothesis)

Additional Diagnostics:
Hurst Exponent: 0.3506 → Mean-reverting
Half-life: 55865.74 days
Variance Ratio p-value: 0.2092


C:\Users\djray\AppData\Local\Temp\ipykernel_41688\4094149535.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\djray\AppData\Local\Temp\ipykernel_41688\4094149535.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [39]:
# backtest a cointegration-based mean reversion strategy

data = pair

# calculate hedge ratio, spread, and z-score
X = sm.add_constant(data['GDX'])
model = sm.OLS(data['GLD'], X).fit()
beta = model.params[1]
spread = data['GLD'] - beta * data['GDX']
z_score = (spread - spread.mean()) / spread.std()

# trading parameters
entry_threshold = 1
exit_threshold = 0.25

# init positions and portfolio
positions = pd.DataFrame(index=data.index)
positions['z'] = z_score
positions['position'] = 0  # 0 = flat, 1 = long spread, -1 = short spread
positions['portfolio_value'] = 100000
positions['cash'] = positions['portfolio_value']
positions['gld_value'] = 0.0
positions['gdx_value'] = 0.0
positions['trade_size'] = 10000  # amount allocated per trade

# use state variables for trading logic
current_position = 0
trade_active = False

for i in range(1, len(positions)):  # trade
    date = positions.index[i]
    prev_date = positions.index[i-1]
    
    # update values
    positions.loc[date, 'cash'] = positions.loc[prev_date, 'cash']
    positions.loc[date, 'gld_value'] = positions.loc[prev_date, 'gld_value']
    positions.loc[date, 'gdx_value'] = positions.loc[prev_date, 'gdx_value']
    positions.loc[date, 'portfolio_value'] = (
        positions.loc[date, 'cash'] + 
        positions.loc[date, 'gld_value'] + 
        positions.loc[date, 'gdx_value']
    )
    
    if trade_active:  # if in position, look for exit conditions
        if current_position == 1:  # long spread position
            if positions.loc[date, 'z'] >= -exit_threshold:
                # exit long spread: sell GLD, buy back GDX
                gld_shares = positions.loc[date, 'gld_value'] / data.loc[date, 'GLD']
                gdx_shares = positions.loc[date, 'gdx_value'] / data.loc[date, 'GDX'] * -1  # Negative value
                
                # close
                cash_from_gld = gld_shares * data.loc[date, 'GLD']
                cash_for_gdx = gdx_shares * data.loc[date, 'GDX'] * -1  # shorted this - mult by -1 for returns
                
                positions.loc[date, 'cash'] += cash_from_gld - cash_for_gdx
                positions.loc[date, 'gld_value'] = 0
                positions.loc[date, 'gdx_value'] = 0
                positions.loc[date, 'position'] = 0
                trade_active = False
                current_position = 0
                
        elif current_position == -1:  # short spread position
            if positions.loc[date, 'z'] <= exit_threshold:
                # exit short spread: buy back GLD, sell GDX
                gld_shares = positions.loc[date, 'gld_value'] / data.loc[date, 'GLD'] * -1  # Negative value
                gdx_shares = positions.loc[date, 'gdx_value'] / data.loc[date, 'GDX']
                
                # close
                cash_for_gld = gld_shares * data.loc[date, 'GLD'] * -1  # shorted this - mult by -1 for returns
                cash_from_gdx = gdx_shares * data.loc[date, 'GDX']
                
                positions.loc[date, 'cash'] += cash_from_gdx - cash_for_gld
                positions.loc[date, 'gld_value'] = 0
                positions.loc[date, 'gdx_value'] = 0
                positions.loc[date, 'position'] = 0
                trade_active = False
                current_position = 0
    
    
    if not trade_active:  # if no active position, look for an entry
        if positions.loc[date, 'z'] < -entry_threshold:
            # long spread: buy GLD, short GDX
            capital = min(positions.loc[date, 'trade_size'], positions.loc[date, 'cash'])
            
            # calculate shares
            gld_shares = (capital / 2) / data.loc[date, 'GLD']
            gdx_shares = (capital / 2) / data.loc[date, 'GDX'] * beta
            
            # execute the trades and update position info
            positions.loc[date, 'cash'] -= gld_shares * data.loc[date, 'GLD']
            positions.loc[date, 'cash'] += gdx_shares * data.loc[date, 'GDX']
            positions.loc[date, 'gld_value'] = gld_shares * data.loc[date, 'GLD']
            positions.loc[date, 'gdx_value'] = -gdx_shares * data.loc[date, 'GDX']  # negative for short position
            
            positions.loc[date, 'position'] = 1
            trade_active = True
            current_position = 1
            
        elif positions.loc[date, 'z'] > entry_threshold:
            # short spread: short GLD, buy GDX
            capital = min(positions.loc[date, 'trade_size'], positions.loc[date, 'cash'])
            
            # calculate shares
            gld_shares = (capital / 2) / data.loc[date, 'GLD']
            gdx_shares = (capital / 2) / data.loc[date, 'GDX'] * beta
            
            # execute the trades and update position info
            positions.loc[date, 'cash'] += gld_shares * data.loc[date, 'GLD']
            positions.loc[date, 'cash'] -= gdx_shares * data.loc[date, 'GDX']
            positions.loc[date, 'gld_value'] = -gld_shares * data.loc[date, 'GLD']  # negative for short position
            positions.loc[date, 'gdx_value'] = gdx_shares * data.loc[date, 'GDX']
            
            positions.loc[date, 'position'] = -1
            trade_active = True
            current_position = -1

positions['portfolio_value'] = positions['cash'] + positions['gld_value'] + positions['gdx_value']

# benchmark returns
benchmark = data['GLD'] / data['GLD'].iloc[0] * 100000

# plot backtest data
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.05,
                    subplot_titles=('Z-Score with Trading Signals', 
                                    'Portfolio Value',
                                    'Position'),
                    row_heights=[0.4, 0.4, 0.2])

fig.add_trace(go.Scatter(x=positions.index, y=positions['z'], 
                         name='Z-Score', line=dict(color='navy')), row=1, col=1)

fig.add_hline(y=0, line=dict(color='black', dash='solid'), row=1, col=1)
fig.add_hline(y=entry_threshold, line=dict(color='red', dash='dash'), row=1, col=1)
fig.add_hline(y=-entry_threshold, line=dict(color='green', dash='dash'), row=1, col=1)
fig.add_hline(y=exit_threshold, line=dict(color='blue', dash='dot'), row=1, col=1)
fig.add_hline(y=-exit_threshold, line=dict(color='blue', dash='dot'), row=1, col=1)

fig.add_hrect(y0=entry_threshold, y1=5, line_width=0, fillcolor="red", opacity=0.1, row=1, col=1)
fig.add_hrect(y0=-entry_threshold, y1=-5, line_width=0, fillcolor="green", opacity=0.1, row=1, col=1)

long_entries = positions[positions['position'] == 1]
short_entries = positions[positions['position'] == -1]
exits = positions[(positions['position'] == 0) & (positions['position'].shift(1) != 0)]

fig.add_trace(go.Scatter(x=long_entries.index, y=long_entries['z'],
                         mode='markers', name='Long Spread Entry',
                         marker=dict(symbol='triangle-up', size=10, color='green')), row=1, col=1)

fig.add_trace(go.Scatter(x=short_entries.index, y=short_entries['z'],
                         mode='markers', name='Short Spread Entry',
                         marker=dict(symbol='triangle-down', size=10, color='red')), row=1, col=1)

fig.add_trace(go.Scatter(x=exits.index, y=exits['z'],
                         mode='markers', name='Exit',
                         marker=dict(symbol='circle', size=8, color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=positions.index, y=positions['portfolio_value'],
                         name='Strategy', line=dict(color='purple', width=2)), row=2, col=1)

fig.add_trace(go.Scatter(x=positions.index, y=benchmark,
                         name='Buy & Hold GLD', line=dict(color='green', width=1)), row=2, col=1)

fig.add_trace(go.Scatter(x=positions.index, y=positions['position'],
                         name='Position', line=dict(color='darkred', width=1.5),
                         fill='tozeroy'), row=3, col=1)

fig.update_layout(
    height=800,
    title_text='GLD-GDX Pair Trading Strategy (2013-2017)',
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

fig.update_yaxes(title_text="Z-Score", row=1, col=1)
fig.update_yaxes(title_text="Portfolio Value ($)", row=2, col=1)
fig.update_yaxes(title_text="Position", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)

fig.update_yaxes(tickvals=[-1, 0, 1], 
                 ticktext=['Short Spread', 'Flat', 'Long Spread'], 
                 row=3, col=1)

cum_return = (positions['portfolio_value'].iloc[-1] / positions['portfolio_value'].iloc[0] - 1) * 100
ann_return = (1 + cum_return/100) ** (252/len(positions)) - 1
volatility = positions['portfolio_value'].pct_change().std() * np.sqrt(252)
sharpe = ann_return / volatility if volatility != 0 else 0

metrics_text = (f"<b>Strategy Performance</b><br>"
                f"Cumulative Return: {cum_return:.1f}%<br>"
                f"Annualized Return: {ann_return:.1%}<br>"
                f"Sharpe Ratio: {sharpe:.2f}")

fig.add_annotation(
    x=0.05, y=0.95,
    xref='paper', yref='paper',
    text=metrics_text,
    showarrow=False,
    bgcolor='white',
    bordercolor='black',
    borderwidth=1,
    row=1, col=1
)

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(go.Scatter(x=spread.index, y=spread,
                          name='Spread (GLD - β*GDX)', line=dict(color='royalblue')),
               secondary_y=False)

fig2.add_trace(go.Scatter(x=z_score.index, y=z_score,
                          name='Z-Score', line=dict(color='darkorange')),
               secondary_y=True)

fig2.update_layout(
    title_text='Spread and Z-Score Analysis',
    template='plotly_white',
    height=500
)

fig2.update_yaxes(title_text="Spread", secondary_y=False)
fig2.update_yaxes(title_text="Z-Score", secondary_y=True)

fig2.add_hline(y=spread.mean(), line=dict(color='royalblue', dash='dash'), secondary_y=False)
fig2.add_hline(y=entry_threshold, line=dict(color='red', dash='dash'), secondary_y=True)
fig2.add_hline(y=-entry_threshold, line=dict(color='green', dash='dash'), secondary_y=True)
fig2.add_hline(y=0, line=dict(color='black', dash='solid'), secondary_y=True)

start_date = '2013-01-01'
end_date = '2017-12-31'
fig.update_xaxes(range=[start_date, end_date])
fig2.update_xaxes(range=[start_date, end_date])

fig.show()
fig2.show()

# calculate performance metrics
def calculate_metrics(portfolio_value):
    returns = portfolio_value.pct_change().fillna(0)
    cumulative_return = (portfolio_value.iloc[-1] / portfolio_value.iloc[0]) - 1
    annualized_return = (1 + cumulative_return) ** (252/len(portfolio_value)) - 1
    volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / volatility if volatility != 0 else np.nan
    
    peak = portfolio_value.cummax()
    drawdown = (portfolio_value - peak) / peak
    max_drawdown = drawdown.min()
    
    return {
        'Cumulative Return': cumulative_return,
        'Annualized Return': annualized_return,
        'Annualized Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }

strategy_perf = calculate_metrics(positions['portfolio_value'])
benchmark_perf = calculate_metrics(benchmark)

print("\nStrategy Performance:")
for metric, value in strategy_perf.items():
    print(f"{metric}: {value:.4f}")

print("\nBenchmark (GLD) Performance:")
for metric, value in benchmark_perf.items():
    print(f"{metric}: {value:.4f}")

C:\Users\djray\AppData\Local\Temp\ipykernel_41688\1950251312.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\djray\AppData\Local\Temp\ipykernel_41688\1950251312.py:109: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '94425.18882784926' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\djray\AppData\Local\Temp\ipykernel_41688\1950251312.py:38: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '141149.6223443015' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.




Strategy Performance:
Cumulative Return: 2.0575
Annualized Return: 0.2507
Annualized Volatility: 0.1462
Sharpe Ratio: 1.7150
Max Drawdown: 0.0000

Benchmark (GLD) Performance:
Cumulative Return: -0.2422
Annualized Return: -0.0540
Annualized Volatility: 0.1578
Sharpe Ratio: -0.3421
Max Drawdown: -0.3860
